In [1]:

from __future__ import annotations
import json
import logging
from abc import ABC, abstractmethod, ABCMeta
from datetime import datetime, timedelta
from functools import wraps
from typing import List, Dict, Any, Optional

# --------------------------- Исключения ---------------------------

class InvalidDateError(Exception):
    """Выбрасывается если дата окончания курса раньше даты начала."""

class PermissionDeniedError(Exception):
    """Выбрасывается если у пользователя нет прав на операцию."""

class CourseNotFoundError(Exception):
    """Курс не найден на платформе."""

# --------------------------- Логирование ---------------------------

logger = logging.getLogger("educational_platform")
logger.setLevel(logging.DEBUG)
# консоль
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
# файл
fh = logging.FileHandler("platform.log", encoding="utf-8")
fh.setLevel(logging.DEBUG)
# форматтер
fmt = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
ch.setFormatter(fmt)
fh.setFormatter(fmt)
# избегаем дублирования хэндлеров при повторном импортировании
if not logger.handlers:
    logger.addHandler(ch)
    logger.addHandler(fh)
else:
    # если обработчики уже есть, заменим их (полезно при интерактивном запуске)
    logger.handlers = [ch, fh]

# --------------------------- Метакласс ---------------------------

class CourseMeta(ABCMeta):
    """Метакласс, регистрирующий подклассы Course для динамического создания по имени типа.
    Наследуется от ABCMeta, чтобы избежать конфликта метаклассов с классами на базе ABC.
    """
    registry: Dict[str, type] = {}

    def __new__(mcls, name, bases, namespace):
        cls = super().__new__(mcls, name, bases, namespace)
        # Регистрируем только реальные подклассы Course (кроме самого Course)
        try:
            if name != 'Course' and issubclass(cls, Course):
                CourseMeta.registry[name.lower()] = cls
                logger.debug(f"Registered course type: {name}")
        except NameError:
            # Course ещё не определён — пропускаем (будет создан позже)
            pass
        return cls

# --------------------------- Интерфейсы (абстрактные классы) ---------------------------

class Teachable(ABC):
    """Интерфейс, описывающий способность преподавать."""

    @abstractmethod
    def teach(self) -> str:
        """Возвращает строку с описанием процесса обучения."""

class Assessable(ABC):
    """Интерфейс для оценки прогресса студентов."""

    @abstractmethod
    def assess_progress(self) -> Dict[str, Any]:
        """Оценивает прогресс и возвращает сводку."""

# --------------------------- Миксины ---------------------------

class LoggingMixin:
    """Добавляет метод логирования действий курса."""

    def log_action(self, message: str) -> None:
        name = getattr(self, 'title', 'Unknown')
        logger.info(f"Курс [{name}]: {message}")

class NotificationMixin:
    """Добавляет метод уведомления студентов (симуляция)."""

    def notify_students(self, message: str) -> None:
        students = getattr(self, 'students', [])
        logger.info(f"Уведомление {len(students)} студентам курса '{getattr(self,'title','?')}': {message}")

# --------------------------- Абстрактный Course (без наследования ABC) ---------------------------

class Course(metaclass=CourseMeta):
    """Базовый абстрактный класс курса.

    ВАЖНО: не наследуем ABC напрямую — используем метакласс CourseMeta (наследник ABCMeta),
    чтобы избежать конфликта метаклассов при множественном наследовании с ABC.
    """

    def __init__(
        self,
        title: str,
        start_date: datetime,
        end_date: datetime,
        instructor: str,
        students: Optional[List[str]] = None,
        topics: Optional[List[str]] = None,
    ):
        if end_date < start_date:
            raise InvalidDateError("Дата окончания раньше даты начала")
        self._title: str = title
        self._start_date: datetime = start_date
        self._end_date: datetime = end_date
        self._instructor: str = instructor
        self._students: List[str] = students or []
        self._topics: List[str] = topics or []

    # Методы, которые делают класс "абстрактным" для целей ABCMeta
    @abstractmethod
    def calculate_completion_rate(self) -> float:
        """Должен возвращать процент завершения курса (0-100)."""
        raise NotImplementedError

    # Геттеры и сеттеры — инкапсуляция
    @property
    def title(self) -> str:
        return self._title

    @title.setter
    def title(self, value: str) -> None:
        self._title = value

    @property
    def start_date(self) -> datetime:
        return self._start_date

    @start_date.setter
    def start_date(self, value: datetime) -> None:
        if self._end_date < value:
            raise InvalidDateError("Дата начала не может быть позже даты окончания")
        self._start_date = value

    @property
    def end_date(self) -> datetime:
        return self._end_date

    @end_date.setter
    def end_date(self, value: datetime) -> None:
        if value < self._start_date:
            raise InvalidDateError("Дата окончания раньше даты начала")
        self._end_date = value

    @property
    def instructor(self) -> str:
        return self._instructor

    @instructor.setter
    def instructor(self, value: str) -> None:
        self._instructor = value

    @property
    def students(self) -> List[str]:
        return self._students

    @students.setter
    def students(self, value: List[str]) -> None:
        self._students = value

    @property
    def topics(self) -> List[str]:
        return self._topics

    @topics.setter
    def topics(self, value: List[str]) -> None:
        self._topics = value

    def duration_days(self) -> int:
        """Длительность курса в днях."""
        return (self._end_date - self._start_date).days + 1

    def __str__(self) -> str:
        return f"Курс: {self._title}, Преподаватель: {self._instructor}"

    # Сравнения: по количеству студентов, затем по длительности
    def __eq__(self, other: object) -> bool:
        if not isinstance(other, Course):
            return NotImplemented
        return len(self._students) == len(other._students)

    def __lt__(self, other: 'Course') -> bool:
        if len(self._students) != len(other._students):
            return len(self._students) < len(other._students)
        return self.duration_days() < other.duration_days()

    def __gt__(self, other: 'Course') -> bool:
        return not (self == other or self < other)

    # Сериализация
    def to_dict(self) -> Dict[str, Any]:
        return {
            'type': type(self).__name__,
            'title': self._title,
            'start_date': self._start_date.isoformat(),
            'end_date': self._end_date.isoformat(),
            'instructor': self._instructor,
            'students': list(self._students),
            'topics': list(self._topics),
        }

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'Course':
        raise NotImplementedError("Используйте CourseFactory.create_course_from_dict")

# --------------------------- Подклассы курсов ---------------------------

class ProgrammingCourse(Course, Teachable, Assessable, LoggingMixin, NotificationMixin):
    """Курс программирования: хранит список языков и прогресс студентов."""

    def __init__(self, languages: List[str], **kwargs):
        super().__init__(**kwargs)
        self._languages = languages
        self._progress: Dict[str, float] = {s: 0.0 for s in self._students}

    @property
    def languages(self) -> List[str]:
        return self._languages

    @languages.setter
    def languages(self, value: List[str]) -> None:
        self._languages = value

    def calculate_completion_rate(self) -> float:
        if not self._students:
            return 0.0
        values = [self._progress.get(s, 0.0) for s in self._students]
        avg = sum(values) / len(values)
        return round(avg, 2)

    def teach(self) -> str:
        return "Провожу лекции по алгоритмам"

    def assess_progress(self) -> Dict[str, Any]:
        for s in self._students:
            self._progress[s] = min(100.0, self._progress.get(s, 0.0) + 10.0)
        result = {s: self._progress[s] for s in self._students}
        return {'average': self.calculate_completion_rate(), 'per_student': result}

    def __str__(self) -> str:
        langs = ', '.join(self._languages)
        return f"Курс программирования: {self._title}, Языки: {langs}"

    def to_dict(self) -> Dict[str, Any]:
        d = super().to_dict()
        d.update({'languages': list(self._languages), 'progress': self._progress})
        return d

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'ProgrammingCourse':
        base = {
            'title': data['title'],
            'start_date': datetime.fromisoformat(data['start_date']),
            'end_date': datetime.fromisoformat(data['end_date']),
            'instructor': data['instructor'],
            'students': data.get('students', []),
            'topics': data.get('topics', []),
        }
        obj = cls(languages=data.get('languages', []), **base)
        obj._progress = data.get('progress', {s: 0.0 for s in obj._students})
        return obj


class DesignCourse(Course, Teachable, Assessable, LoggingMixin, NotificationMixin):
    """Курс дизайна: хранит инструменты и оценки студентов."""

    def __init__(self, tools: List[str], **kwargs):
        super().__init__(**kwargs)
        self._tools = tools
        self._grades: Dict[str, float] = {s: 0.0 for s in self._students}

    @property
    def tools(self) -> List[str]:
        return self._tools

    @tools.setter
    def tools(self, value: List[str]) -> None:
        self._tools = value

    def calculate_completion_rate(self) -> float:
        if not self._students:
            return 0.0
        avg = sum(self._grades.values()) / len(self._students)
        return round(avg, 2)

    def teach(self) -> str:
        return "Объясняю принципы композиции"

    def assess_progress(self) -> Dict[str, Any]:
        for s in self._students:
            self._grades[s] = min(100.0, self._grades.get(s, 0.0) + 5.0)
        return {'average': self.calculate_completion_rate(), 'per_student': self._grades}

    def __str__(self) -> str:
        return f"Курс дизайна: {self._title}, Инструменты: {', '.join(self._tools)}"

    def to_dict(self) -> Dict[str, Any]:
        d = super().to_dict()
        d.update({'tools': list(self._tools), 'grades': self._grades})
        return d

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'DesignCourse':
        base = {
            'title': data['title'],
            'start_date': datetime.fromisoformat(data['start_date']),
            'end_date': datetime.fromisoformat(data['end_date']),
            'instructor': data['instructor'],
            'students': data.get('students', []),
            'topics': data.get('topics', []),
        }
        obj = cls(tools=data.get('tools', []), **base)
        obj._grades = data.get('grades', {s: 0.0 for s in obj._students})
        return obj


class ScienceCourse(Course, Teachable, Assessable, LoggingMixin, NotificationMixin):
    """Курс наук: хранит область науки и выполнение лабораторных."""

    def __init__(self, field: str, **kwargs):
        super().__init__(**kwargs)
        self._field = field
        self._lab_completion: Dict[str, int] = {s: 0 for s in self._students}

    @property
    def field(self) -> str:
        return self._field

    @field.setter
    def field(self, value: str) -> None:
        self._field = value

    def calculate_completion_rate(self) -> float:
        if not self._students:
            return 0.0
        values = [(self._lab_completion.get(s, 0)/10)*100 for s in self._students]
        avg = sum(values)/len(values)
        return round(avg, 2)

    def teach(self) -> str:
        return "Провожу лабораторные работы"

    def assess_progress(self) -> Dict[str, Any]:
        for s in self._students:
            self._lab_completion[s] = min(10, self._lab_completion.get(s, 0) + 1)
        return {'average': self.calculate_completion_rate(), 'per_student': self._lab_completion}

    def __str__(self) -> str:
        return f"Курс наук: {self._title}, Область: {self._field}"

    def to_dict(self) -> Dict[str, Any]:
        d = super().to_dict()
        d.update({'field': self._field, 'lab_completion': self._lab_completion})
        return d

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'ScienceCourse':
        base = {
            'title': data['title'],
            'start_date': datetime.fromisoformat(data['start_date']),
            'end_date': datetime.fromisoformat(data['end_date']),
            'instructor': data['instructor'],
            'students': data.get('students', []),
            'topics': data.get('topics', []),
        }
        obj = cls(field=data.get('field', ''), **base)
        obj._lab_completion = data.get('lab_completion', {s: 0 for s in obj._students})
        return obj

# --------------------------- Фабрика ---------------------------

class CourseFactory:
    @staticmethod
    def create_course(kind: str, **kwargs) -> Course:
        """Создает курс по типу (programming|design|science) или по имени класса."""
        key = kind.lower()
        mapping = {
            'programming': ProgrammingCourse,
            'design': DesignCourse,
            'science': ScienceCourse,
        }
        cls = mapping.get(key) or CourseMeta.registry.get(key) or CourseMeta.registry.get(f"{key}course")
        if cls is None:
            raise ValueError(f"Неизвестный тип курса: {kind}")
        return cls(**kwargs)

    @staticmethod
    def create_course_from_dict(data: Dict[str, Any]) -> Course:
        t = data.get('type', '').lower()
        # Пробуем явно по типу
        if 'program' in t:
            return ProgrammingCourse.from_dict(data)
        if 'design' in t:
            return DesignCourse.from_dict(data)
        if 'science' in t:
            return ScienceCourse.from_dict(data)
        # fallback: попытка найти по реестру
        cls = CourseMeta.registry.get(t)
        if cls and hasattr(cls, 'from_dict'):
            return cls.from_dict(data)
        raise ValueError('Не удалось определить тип курса из данных')

# --------------------------- Композиция: Address и Platform ---------------------------

class Address:
    def __init__(self, city: str, street: str, number: str):
        self.city = city
        self.street = street
        self.number = number

    def __str__(self) -> str:
        return f"{self.city}, {self.street} {self.number}"

class Platform:
    def __init__(self, name: str, address: Address):
        self.name = name
        self.address = address
        self._courses: List[Course] = []

    def add_course(self, course: Course) -> None:
        self._courses.append(course)
        logger.info(f"Курс добавлен: {course.title}")

    def remove_course(self, title: str) -> None:
        for i, c in enumerate(self._courses):
            if c.title == title:
                removed = self._courses.pop(i)
                logger.info(f"Курс удален: {title}")
                return
        raise CourseNotFoundError(f"Курс с названием '{title}' не найден")

    def get_courses(self) -> List[Course]:
        return list(self._courses)

    def find_course(self, title: str) -> Course:
        for c in self._courses:
            if c.title == title:
                return c
        raise CourseNotFoundError(title)

    def get_top_courses(self, n: int = 3) -> List[Course]:
        return sorted(self._courses, key=lambda c: len(c.students), reverse=True)[:n]

    def to_dict(self) -> Dict[str, Any]:
        return {'name': self.name, 'address': {'city': self.address.city, 'street': self.address.street, 'number': self.address.number}, 'courses': [c.to_dict() for c in self._courses]}

    def save_to_file(self, filename: str) -> None:
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(self.to_dict(), f, ensure_ascii=False, indent=2)
        logger.info(f"Платформа сохранена в {filename}")

    @staticmethod
    def load_from_file(filename: str) -> 'Platform':
        with open(filename, 'r', encoding='utf-8') as f:
            data = json.load(f)
        addr = data.get('address', {})
        p = Platform(data.get('name', 'Unknown'), Address(addr.get('city',''), addr.get('street',''), addr.get('number','')))
        for cdata in data.get('courses', []):
            course = CourseFactory.create_course_from_dict(cdata)
            p.add_course(course)
        return p

# --------------------------- Декоратор прав доступа ---------------------------

def check_permissions(required_role: str):
    """Декоратор проверяет, что в kwargs есть user с нужной ролью."""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            user = kwargs.get('user')
            if not user or required_role not in user.get('roles', []):
                logger.warning(f"Доступ запрещен для пользователя {user}")
                raise PermissionDeniedError(f"Пользователь не имеет роль '{required_role}'")
            return func(*args, **kwargs)
        return wrapper
    return decorator

# --------------------------- Цепочка обязанностей ---------------------------

class ChangeRequest:
    def __init__(self, course_title: str, change_type: str, details: str):
        self.course_title = course_title
        self.change_type = change_type
        self.details = details

class Handler(ABC):
    def __init__(self, successor: Optional['Handler'] = None):
        self._successor = successor

    @abstractmethod
    def handle(self, request: ChangeRequest) -> bool:
        pass

class InstructorHandler(Handler):
    def handle(self, request: ChangeRequest) -> bool:
        if request.change_type == 'material':
            logger.info(f"Instructor approved materials change for {request.course_title}")
            return True
        if self._successor:
            return self._successor.handle(request)
        return False

class MethodologyDepartmentHandler(Handler):
    def handle(self, request: ChangeRequest) -> bool:
        if request.change_type == 'structure':
            logger.info(f"Methodology dept approved structure change for {request.course_title}")
            return True
        if self._successor:
            return self._successor.handle(request)
        return False

class ManagementHandler(Handler):
    def handle(self, request: ChangeRequest) -> bool:
        logger.info(f"Management approved change for {request.course_title}")
        return True

# --------------------------- Шаблонный метод (ProgressAssessor) ---------------------------

class ProgressAssessor(ABC):
    def assess_progress(self, course: Course) -> Dict[str, Any]:
        results = self.get_current_results(course)
        average = self.compute_average(results)
        applied = self.apply_assessment(course, results, average)
        return {'average': average, 'applied': applied}

    @abstractmethod
    def get_current_results(self, course: Course) -> Dict[str, float]:
        pass

    @abstractmethod
    def compute_average(self, results: Dict[str, float]) -> float:
        pass

    @abstractmethod
    def apply_assessment(self, course: Course, results: Dict[str, float], average: float) -> Any:
        pass

class ProgrammingProgressAssessor(ProgressAssessor):
    def get_current_results(self, course: ProgrammingCourse) -> Dict[str, float]:
        return getattr(course, '_progress', {s: 0.0 for s in course.students})

    def compute_average(self, results: Dict[str, float]) -> float:
        if not results:
            return 0.0
        return round(sum(results.values())/len(results), 2)

    def apply_assessment(self, course: ProgrammingCourse, results: Dict[str, float], average: float) -> Any:
        return {'course': course.title, 'count': len(results)}

class DesignProgressAssessor(ProgressAssessor):
    def get_current_results(self, course: DesignCourse) -> Dict[str, float]:
        return getattr(course, '_grades', {s: 0.0 for s in course.students})

    def compute_average(self, results: Dict[str, float]) -> float:
        if not results:
            return 0.0
        return round(sum(results.values())/len(results), 2)

    def apply_assessment(self, course: DesignCourse, results: Dict[str, float], average: float) -> Any:
        return {'course': course.title, 'noted_students': [s for s, g in results.items() if g < 50]}

# --------------------------- Демонстрация и тесты ---------------------------

if __name__ == '__main__':
    # Простые тесты и демонстрация — их можно расширять.
    addr = Address('Sofia', 'Tsarigradsko', '15')
    platform = Platform('EduPlatform', addr)

    start = datetime.now()
    end = start + timedelta(days=30)

    # Создаём курсы через фабрику
    prog = CourseFactory.create_course('programming', title='Intro to Python', start_date=start, end_date=end, instructor='Ivan Petrov', students=['a','b','c'], topics=['syntax','OOP'], languages=['Python','JavaScript'])
    design = CourseFactory.create_course('design', title='Basics of Design', start_date=start, end_date=end, instructor='Maria Ivanova', students=['d','e'], topics=['color','composition'], tools=['Photoshop','Figma'])
    science = CourseFactory.create_course('science', title='Physics 101', start_date=start, end_date=end, instructor='Dr. Strange', students=['x','y','z','w'], topics=['mechanics'], field='Physics')

    platform.add_course(prog)
    platform.add_course(design)
    platform.add_course(science)

    # Проверки реестра метакласса
    assert 'programmingcourse' in CourseMeta.registry, f"programmingcourse not registered: {CourseMeta.registry.keys()}"
    assert 'designcourse' in CourseMeta.registry
    assert 'sciencecourse' in CourseMeta.registry

    # Логирование и уведомления (симуляция)
    prog.log_action('Курс создан и опубликован')
    design.notify_students('Новый модуль доступен')

    # Teach и оценка
    print(prog.teach())
    print(design.teach())
    print(science.teach())

    print('Completion (before):', prog.calculate_completion_rate())
    assessed = prog.assess_progress()
    print('Assess result:', assessed)
    print('Completion (after):', prog.calculate_completion_rate())

    # Топ курсов
    top = platform.get_top_courses(2)
    print('Top courses by students:')
    for c in top:
        print('-', c.title, 'students:', len(c.students))

    # Цепочка обязанностей
    handler_chain = InstructorHandler(MethodologyDepartmentHandler(ManagementHandler()))
    req = ChangeRequest('Intro to Python', 'structure', 'Добавить модуль о тестировании')
    handled = handler_chain.handle(req)
    print('Change handled:', handled)

    # Шаблонный метод
    assessor = ProgrammingProgressAssessor()
    print('Assessor result:', assessor.assess_progress(prog))

    # Сериализация и десериализация
    fname = 'platform_data.json'
    platform.save_to_file(fname)
    loaded = Platform.load_from_file(fname)
    assert len(loaded.get_courses()) == len(platform.get_courses())
    print('Loaded courses count OK')

    # Проверка декоратора прав
    @check_permissions('admin')
    def change_course_title(platform: Platform, old_title: str, new_title: str, user: Dict[str, Any]):
        c = platform.find_course(old_title)
        c.title = new_title
        return c

    admin = {'name': 'Boss', 'roles': ['admin', 'manager']}
    non_admin = {'name': 'Guest', 'roles': ['student']}

    changed = change_course_title(platform, 'Intro to Python', 'Python 101', user=admin)
    assert changed.title == 'Python 101'
    print('Title change by admin OK')

    try:
        change_course_title(platform, 'Python 101', 'Should fail', user=non_admin)
    except PermissionDeniedError:
        print('Permission check correctly blocked non-admin')

    # Ошибки при некорректных датах
    try:
        CourseFactory.create_course('programming', title='Bad dates', start_date=end, end_date=start - timedelta(days=1), instructor='X', students=[], topics=[], languages=['Py'])
    except InvalidDateError as e:
        logger.error(f'Invalid date caught as expected: {e}')

    logger.info('All demo checks finished')
    print('Done')


DEBUG:educational_platform:Registered course type: ProgrammingCourse
DEBUG:educational_platform:Registered course type: DesignCourse
DEBUG:educational_platform:Registered course type: ScienceCourse
2025-09-17 10:38:15,319 - INFO - Курс добавлен: Intro to Python
INFO:educational_platform:Курс добавлен: Intro to Python
2025-09-17 10:38:15,322 - INFO - Курс добавлен: Basics of Design
INFO:educational_platform:Курс добавлен: Basics of Design
2025-09-17 10:38:15,324 - INFO - Курс добавлен: Physics 101
INFO:educational_platform:Курс добавлен: Physics 101
2025-09-17 10:38:15,326 - INFO - Курс [Intro to Python]: Курс создан и опубликован
INFO:educational_platform:Курс [Intro to Python]: Курс создан и опубликован
2025-09-17 10:38:15,327 - INFO - Уведомление 2 студентам курса 'Basics of Design': Новый модуль доступен
INFO:educational_platform:Уведомление 2 студентам курса 'Basics of Design': Новый модуль доступен
2025-09-17 10:38:15,329 - INFO - Methodology dept approved structure change for Int

Провожу лекции по алгоритмам
Объясняю принципы композиции
Провожу лабораторные работы
Completion (before): 0.0
Assess result: {'average': 10.0, 'per_student': {'a': 10.0, 'b': 10.0, 'c': 10.0}}
Completion (after): 10.0
Top courses by students:
- Physics 101 students: 4
- Intro to Python students: 3
Change handled: True
Assessor result: {'average': 10.0, 'applied': {'course': 'Intro to Python', 'count': 3}}
Loaded courses count OK
Title change by admin OK
Permission check correctly blocked non-admin
Done
